In [1]:
import numpy

import keras
import keras.backend as K
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from collections import namedtuple
from keract import get_activations
from sklearn.model_selection import train_test_split

import numpy as np
import os
import _pickle as cPickle

import utils

cfg = {}
cfg['SGD_BATCHSIZE']    = 64
cfg['NUM_EPOCHS']       = 1000
cfg['FULL_MI']          = True
cfg['ACTIVATION'] = 'relu'
cfg['SAVE_DIR'] = 'rawdata/lstm_model'

ARCH_NAME =  'lstm'

nb_classes = 2

# fix random seed for reproducibility
numpy.random.seed(10)
# load the dataset but only keep the top n words, zero the rest
top_words = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# truncate and pad input sequences
max_review_length = 256
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

Y_train = keras.utils.np_utils.to_categorical(y_train, nb_classes).astype('float32')
Y_test  = keras.utils.np_utils.to_categorical(y_test, nb_classes).astype('float32')

Dataset = namedtuple('Dataset',['X','Y','y','nb_classes'])
trn = Dataset(X_train, Y_train, y_train, nb_classes)
tst = Dataset(X_test , Y_test, y_test, nb_classes)

# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(64, activation='tanh'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

if not os.path.exists(cfg['SAVE_DIR']):
            print("Making directory", cfg['SAVE_DIR'])
            os.makedirs(cfg['SAVE_DIR'])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 256, 32)           3200      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 28,162
Trainable params: 28,162
Non-trainable params: 0
_________________________________________________________________
None


In [2]:
checkpoint = [keras.callbacks.ModelCheckpoint(filepath='rawdata/lstm_model/model_{epoch:04d}.hdf5', save_weights_only=False)]
r = model.fit(X_train, Y_train, 
              batch_size = cfg['SGD_BATCHSIZE'],
              epochs = cfg['NUM_EPOCHS'],
              validation_split = 0.2,
              shuffle = True,
              # validation_data=(tst.X, tst.Y),
              callbacks = checkpoint)

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 276s 14ms/step - loss: 0.6360 - acc: 0.6337 - val_loss: 0.6441 - val_acc: 0.6192
Epoch 2/2
20000/20000 [==============================] - 279s 14ms/step - loss: 0.5923 - acc: 0.6924 - val_loss: 0.5775 - val_acc: 0.7068


In [ ]:
import matplotlib.pyplot as plt

NUM_EPOCHS = cfg['NUM_EPOCHS']
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

# summarize history for accuracy
axes[0].plot(range(1,NUM_EPOCHS+1), r.history['acc'], '-x')
axes[0].plot(range(1,NUM_EPOCHS+1), r.history['val_acc'], '-+')
axes[0].set_title('Model accuracy')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_xticks(range(1,NUM_EPOCHS+1))
axes[0].grid(linestyle='-')
axes[0].legend(['train', 'validation'], loc='upper left')

# summarize history for loss
axes[1].plot(range(1,NUM_EPOCHS+1), r.history['loss'], '-x')
axes[1].plot(range(1,NUM_EPOCHS+1), r.history['val_loss'], '-+')
axes[1].set_title('Model loss')
axes[1].set_ylabel('Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_xticks(range(1,NUM_EPOCHS+1))
axes[1].grid(linestyle='-')
axes[1].legend(['train', 'validation'], loc='upper left')

In [3]:
cur_dir = cfg['SAVE_DIR']
i = 0
for epochfile in sorted(os.listdir(cur_dir)):
    fname = cur_dir + "/" + epochfile
    model = keras.models.load_model(fname)
    
    data = {
            'activity_tst' : []    # Activity in each layer for test set
            }
    
    for layer in model.layers:
        if layer.name == 'embedding_1':
            continue
        
        intermediate_layer_model = keras.models.Model(inputs=model.input, outputs=model.get_layer(layer.name).output)
        intermediate_output = intermediate_layer_model.predict(X_train)  
            
        data['activity_tst'].append(intermediate_output)
        
        del intermediate_layer_model
        
    fname = 'rawdata/relu_lstm' + "/epoch%08d"% i
    print("Saving", fname)
    with open(fname, 'wb') as f:
        cPickle.dump({'ACTIVATION':cfg['ACTIVATION'], 'epoch':i, 'data':data}, f)   
    i += 1
    
    del model

Saving rawdata/relu_lstm/epoch00000000
Saving rawdata/relu_lstm/epoch00000001
